In [1]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import pymongo

In [2]:
# Initialize PyMongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.items

In [4]:
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [8]:
#Soup (object)
news_soup = bs(response.text, 'lxml')
news = news_soup.find('div', class_ = 'content_title').text.strip()
print(news)

NASA's Opportunity Rover Mission on Mars Comes to End


In [12]:
newsHL = news_soup.find('div', class_ = 'rollover_description_inner').text.strip()
print(newsHL)

NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.


In [15]:
from selenium import webdriver
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [14]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
browser.click_link_by_id('full_image')
time.sleep(2)

In [13]:
#Image
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [16]:
featured_img_url = "https://www.jpl.nasa.gov" + img_url
featured_img_url

In [20]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

In [23]:
weather_soup = bs(response.text, 'html.parser')
mars_weather = weather_soup.find_all('div', class_ = "js-tweet-text-container")
for tweet in mars_weather_tweet:
    if tweet.text.strip().startswith('Sol'):
        weather = tweet.text.strip()
print(weather)

Sol 2306 (2019-01-31), high -12C/10F, low -70C/-93F, pressure at 8.13 hPa, daylight 06:47-18:54


In [25]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [26]:
# find the html table and convert to pandas dataframe
mars_df = pd.read_html(url)
mars_df = (mars_df[0])

In [29]:
#set column headers 
mars_df.columns = ["Description", "Value"]
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [30]:
mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n', ' ')
table_data

'<table border="1" class="dataframe mars">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Description</th>       <th>Value</th>     </tr>   </thead>   <tbody>     <tr>       <th>0</th>       <td>Equatorial Diameter:</td>       <td>6,792 km</td>     </tr>     <tr>       <th>1</th>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <th>2</th>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>3</th>       <td>Moons:</td>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>4</th>       <td>Orbit Distance:</td>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>5</th>       <td>Orbit Period:</td>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>6</th>       <td>Surface Temperature:</td>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>7</th>       <td>First Record:</td>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>8</t

In [32]:
#dictionary to be inserted as a MongoDB document
post = {
    'headline': headline, 
    'news': news, 
    'featured': featured_img_url,
    'weather': mars_weather,
    'facts': table_data
}
collection.insert_one(post)

In [33]:
webdatas = db.items.find()

for webdata in webdatas:
    print(webdata)